In [1]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK, PULP_CBC_CMD

__Part 1.__

The Bavarian Motor Company (BMC) manufactures expensive luxury cars in Hamburg, Germany, and exports cars to sell in the United States. The exported cars are shipped from Hamburg to ports in Newark, New Jersey and Jacksonville, Florida. From these ports, the cars are transported by rail or truck to distributors located in Boston, Massachusetts; Columbus, Ohio; Atlanta, Georgia; Richmond, Virginia; and Mobile, Alabama. The below figure shows the possible shipping routes available to the company along with the transportation cost for shipping each car along the indicated path. Currently, 200 cars are available at the port in Newark and 300 are available in Jacksonville. The numbers of cars needed by the distributors in Boston, Columbus, Atlanta, Richmond, and Mobile are 100, 60, 170, 80, and 70, respectively. BMC wants to determine the least costly way of transporting cars from the ports in Newark and Jacksonville to the cities where they are needed.  

1. Formulate the LP and solve it using software of your choice. 
2. Conduct sensitivity analysis and interpret.  
3. Provide the duals and demonstrate strong duality (i.e., complementary slackness).

Strong duality, asserts that the minimal cost in the dual equals the maximal profit in the primal. (https://web.mit.edu/15.053/www/AMP-Chapter-04.pdf)

![alt text](network.png "BMC")

In [40]:
# Problem (BMC Problem)
# define variables
newark_port_to_boston_distributor = LpVariable("newark_port_transport_to_boston_distributor", 0, None)
newark_port_to_richmond_distributor = LpVariable("newark_port_transport_to_richmond_distributor", 0, None)
jacksonville_port_to_richmond_distributor = LpVariable("jacksonville_port_transport_to_richmond_distributor", 0, None)
jacksonville_port_to_mobile_distributor = LpVariable("jacksonville_port_transport_to_mobile_distributor", 0, None)
jacksonville_port_to_atlanta_distributor = LpVariable("jacksonville_port_transport_to_atlanta_distributor", 0, None)

boston_distributor_to_columbus_distributor = LpVariable("boston_distributor_transport_to_columbus_distributor", 0, None)
atlanta_distributor_to_richmond_distributor = LpVariable("atlanta_distributor_transport_to_richmond_distributor", 0, None)
atlanta_distributor_to_mobile_distributor = LpVariable("atlanta_distributor_transport_to_mobile_distributor", 0, None)
mobile_distributor_to_atlanta_distributor = LpVariable("mobile_distributor_transport_to_atlanta_distributor", 0, None)
atlanta_distributor_to_columbus_distributor = LpVariable("atlanta_distributor_transport_to_columbus_distributor", 0, None)
columbus_distributor_to_atlanta_distributor = LpVariable("columbus_distributor_transport_to_atlanta_distributor", 0, None)

# Total supply is 20 cars greater than demand at distributors
#newark_port_to_dummy_distributor = LpVariable("newark_port_transport_to_dummy_distributor", 0, None)
#jacksonville_port_to_dummy_distributor = LpVariable("jacksonville_port_transport_to_dummy_distributor", 0, None)

# defines the problem
bmc_problem = LpProblem("BMC Transport Problem", LpMinimize)

# define objective function
bmc_problem += (30*newark_port_to_boston_distributor + 
                40*newark_port_to_richmond_distributor + 
                50*jacksonville_port_to_richmond_distributor +
                50*jacksonville_port_to_mobile_distributor + 
                45*jacksonville_port_to_atlanta_distributor + 
                50*boston_distributor_to_columbus_distributor + 
                30*atlanta_distributor_to_richmond_distributor + 
                35*atlanta_distributor_to_mobile_distributor + 
                25*mobile_distributor_to_atlanta_distributor + 
                40*atlanta_distributor_to_columbus_distributor +
                35*columbus_distributor_to_atlanta_distributor)

# define constraints
# The numbers of cars needed by the distributors in 
# Boston, Columbus, Atlanta, Richmond, and Mobile are 100, 60, 170, 80, and 70, respectively

bmc_problem += (newark_port_to_boston_distributor - boston_distributor_to_columbus_distributor >= 100, 
                '_100 Cars are Needed in Boston')

bmc_problem += (boston_distributor_to_columbus_distributor + atlanta_distributor_to_columbus_distributor -
                columbus_distributor_to_atlanta_distributor >= 60, 
                '_60 Cars are Needed in Columbus')

bmc_problem += (jacksonville_port_to_atlanta_distributor + mobile_distributor_to_atlanta_distributor -
                atlanta_distributor_to_richmond_distributor - atlanta_distributor_to_columbus_distributor - 
                atlanta_distributor_to_mobile_distributor >= 170, 
                '_170 Cars are Needed in Atlanta')

bmc_problem += (jacksonville_port_to_richmond_distributor + newark_port_to_richmond_distributor +
                atlanta_distributor_to_richmond_distributor >= 80, 
                '_80 Cars are Needed in Richmond')

bmc_problem += (jacksonville_port_to_mobile_distributor + atlanta_distributor_to_mobile_distributor -
                mobile_distributor_to_atlanta_distributor >= 70, 
                '_70 Cars are Needed in Mobile')

# Currently, 200 cars are available at the port in Newark and 300 are available in Jacksonville
bmc_problem += (- newark_port_to_boston_distributor - newark_port_to_richmond_distributor >= -200, 
                '_200 cars are available at the port in Newark')

bmc_problem += (- jacksonville_port_to_mobile_distributor - jacksonville_port_to_richmond_distributor - 
                jacksonville_port_to_atlanta_distributor >= -300, 
                '_300 cars are available in Jacksonville')

## Greater than 0 constraints
#for v in bmc_problem.variables():
#    bmc_problem += (v >= 0, f"{v.name} Non-negative Constraint")

# solve the problem
bmc_problem.writeLP("bmc_problem.lp")

## With GLPK
bmc_problem.solve(GLPK(msg=0, options=['--ranges', 'bmc_problem.sen']))
print("Status:", LpStatus[bmc_problem.status])

# Note, we are only able to get sensitivity information because we are solving
# as a linear program.  If we solved as an Integer Program, then no 
# sensitivity information would be available.
print(f"Minimum cost: {'${:,.2f}'.format(value(bmc_problem.objective))}")  ## formats automatically with commas at thou
for v in bmc_problem.variables():
    print(f'Number of {v.name} = {v.varValue + 0}') # + 0 ensures no negative zeros
print ("")

f = open("bmc_problem.sen", "r")
print(f.read())

Status: Optimal
Minimum cost: $22,350.00
Number of atlanta_distributor_transport_to_columbus_distributor = 40.0
Number of atlanta_distributor_transport_to_mobile_distributor = 0.0
Number of atlanta_distributor_transport_to_richmond_distributor = 0.0
Number of boston_distributor_transport_to_columbus_distributor = 20.0
Number of columbus_distributor_transport_to_atlanta_distributor = 0.0
Number of jacksonville_port_transport_to_atlanta_distributor = 210.0
Number of jacksonville_port_transport_to_mobile_distributor = 70.0
Number of jacksonville_port_transport_to_richmond_distributor = 0.0
Number of mobile_distributor_transport_to_atlanta_distributor = 0.0
Number of newark_port_transport_to_boston_distributor = 120.0
Number of newark_port_transport_to_richmond_distributor = 80.0

GLPK 5.0  - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  OBJ = 22350 (MINimum)

   No. Row name     St      Activity      

/Users/robords/.venv/basic_linear_programming/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


__Conduct sensitivity analysis and interpret__

In general, the sensitivity analysis tells us how the solution changes when you change the coefficients of the model.  In the case of this problem, the second sections covers the variables, and it's first telling us, for example, how much to transport from Atlanta to Columbus (`atlanta_distributor_transport_to_columbus_distributor`), 40.  This matches the output of the variables from the model, so no surprise there. It's telling us we could, all things being equal, transport between 35 and 45 cars on this path, and the objective coefficient will not change. Some of the values here are interesting, i.e., for `columbus_distributor_transport_to_atlanta_distributor`, which can range from -85 cars to an unbounded many without impacting the objective coefficient. 

Staying in Columbis, for the constraint `_60_Cars_are_Needed_in_Columbus`, the shadow price is interesting since it's the largest shadow price. If we changed how many cars were needed in Columbus by one unit, the objective coefficient would increase by \\$85.   This shadow price is only valid between 20 and 80 units. It being the largest, this implies that, all things being equal, this is where increased demand (i.e. if the distributor wanted more units), would be ideal to satisfy first.  On the opposite end of the spectrum is Newark, with a -5 shadow price.  This means, that with one more unit (up to 240, in this case), the optimal solution will decrease by \\$5.  This seems to indicate that it's more worthwhile to land more cars in the Jacksonville port.

In [49]:
# Dual Problem (BMC Problem)

y1 = LpVariable("y1", 0, None)
y2 = LpVariable("y2", 0, None)
y3 = LpVariable("y3", 0, None)
y4 = LpVariable("y4", 0, None)
y5 = LpVariable("y5", 0, None)
y6 = LpVariable("y6", 0, None)
y7 = LpVariable("y7", 0, None)

# defines the problem
bmc_problem_dual = LpProblem("BMC Transport Problem Dual", LpMaximize)

# define objective function
bmc_problem_dual += (60*y1 + 100*y2 + 170*y3 + 80*y4 + 70*y5 - 200*y6 - 300*y7)

################

bmc_problem_dual += (y1 - y6 <= 30, 'dual_constraint_1')
bmc_problem_dual += (- y1 + y2 <= 40, 'dual_constraint_2')
bmc_problem_dual += (y2 - y3 <= 50, 'dual_constraint_3')
bmc_problem_dual += (-y2 <= 50, 'dual_constraint_4')
bmc_problem_dual += (y3 - y7 <= 45, 'dual_constraint_5')
bmc_problem_dual += (y3 - y5 <= 50, 'dual_constraint_6')
bmc_problem_dual += (- y3 + y4 <= 30, 'dual_constraint_7')
bmc_problem_dual += (- y3 + y5 <= 35, 'dual_constraint_8')
bmc_problem_dual += (y4 - y7 <= 25, 'dual_constraint_9')
bmc_problem_dual += (y4 - y6 <= 40, 'dual_constraint_10')
bmc_problem_dual += (y5 - y7 <= 35, 'dual_constraint_11')


/Users/robords/.venv/basic_linear_programming/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Provide the duals and demonstrate strong duality (i.e., complementary slackness).

Strong duality says the sum of the variables in the primal equal the sum of the RHS side parameters in the dual.  In this case, the sum of the variables in the primal are: 30+40+50+50+45+50+30+35+25+40+35 = 430. The sum of the RHS of the constraints in the dual are the same set of numbers, since we derive the RHS of the constraints in the dual from the values of the variables in the objective function.


__Part 2.  A property of strong duality (complementary slackness) is that the sum product of costs X variables in the primal equal the sum product of RHS parameters X duals.  Under what circumstances does this NOT hold?__

This doesn't hold when the primal is unbounded.  If the primal is unbounded, then the dual must be infeasible.  Since strong duality says the primal and dual objective functions at an optimum are equal, then if the sum product of the variables in the primal do not equal the sum of the RHS side parameters, then one solution must be infeasible, and the other unbounded.
